In [18]:
import time
import os

import pandas as pd
import numpy as np
import librosa

from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from pydub.utils import mediainfo

from monitoring import *
from audio import *

In [19]:
taxonomy = pd.read_csv('data/taxonomy.csv')
df = pd.read_csv('data/train.csv')

In [20]:
taxonomy

,primary_label,inat_taxon_id,scientific_name,common_name,class_name
0,1139490,1139490,Ragoniella pulchella,Ragoniella pulchella,Insecta
1,1192948,1192948,Oxyprora surinamensis,Oxyprora surinamensis,Insecta
2,1194042,1194042,Copiphora colombiae,Copiphora colombiae,Insecta
3,126247,126247,Leptodactylus insularum,Spotted Foam-nest Frog,Amphibia
4,1346504,1346504,Neoconocephalus brachypterus,Neoconocephalus brachypterus,Insecta
...,...,...,...,...,...
201,yehcar1,1432779,Milvago chimachima,Yellow-headed Caracara,Aves
202,yelori1,9352,Icterus nigrogularis,Yellow Oriole,Aves
203,yeofly1,16567,Tolmomyias sulphurescens,Yellow-olive Flycatcher,Aves
204,yercac1,10359,Cacicus cela,Yellow-rumped Cacique,Aves


In [21]:
print(df.columns)
df

Index(['primary_label', 'secondary_labels', 'type', 'filename', 'collection',
       'rating', 'url', 'latitude', 'longitude', 'scientific_name',
       'common_name', 'author', 'license'],
      dtype='object')


,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,Yellow-crowned Parrot,Yennifer Alfaro,cc-by-nc 4.0
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,Yellow-crowned Parrot,Evan Centanni,cc-by-nc-sa 4.0
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,Yellow-crowned Parrot,Henrry,cc-by 4.0
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,Yellow-crowned Parrot,Alejandro Luy,cc-by-nc 4.0


From here i am going to build a basic function that opens a given file from a filename and extracts information about it from the metadata of the file.

In [22]:
def get_audio_metadata(filename: str) -> dict:
    """
    Extract audio metadata from a file.

    Args:
        filename (str): Path to the audio file as listed in the dataframe.

    Returns:
        dict: A dictionary containing the following metadata:
            - sample_rate (str): The sample rate of the audio file.
            - bit_rate (str): The bit rate of the audio file.
            - sample_fmt (str): The sample format of the audio file.
            - duration (int): The duration of the audio file in milliseconds.
            - num_channels (int): The number of channels in the audio file.
            - file_codec (str): The codec used in the audio file.
            - file_codec_long_name (str): The long name of the codec.
            - array_size (int): The size of the audio sample array for a single channel.
    """
    filename = 'data/train_audio/' + filename
    info = mediainfo(filename)
    sound = AudioSegment.from_file(filename)
    array = np.array(sound.get_array_of_samples())
    meta_data = {
        'sample_rate': info['sample_rate'],
        'bit_rate': info['bit_rate'],
        'sample_fmt': info['sample_fmt'],
        'duration': len(sound),
        'num_channels': sound.channels,
        'file_codec': info['codec_name'],
        'file_codec_long_name': info['codec_long_name'],
        'array_size': len(array)
    }
    return meta_data


In [23]:

get_audio_metadata(df.iloc[0]['filename'])

{'sample_rate': '32000',
 'bit_rate': '62385',
 'sample_fmt': 'fltp',
 'duration': 98853,
 'num_channels': 1,
 'file_codec': 'vorbis',
 'file_codec_long_name': 'Vorbis',
 'array_size': 3163308}

In [24]:
display = None
for i in range(10):
    time.sleep(0.1)
    display = print_progress_bar(i+1, 10, "test", display_handler=display)

In [25]:
columns = df.columns.tolist()
columns.append('sample_rate')
columns.append('bit_rate')
columns.append('sample_fmt')
columns.append('duration')
columns.append('num_channels')
columns.append('file_codec')
columns.append('file_codec_long_name')
columns.append('array_size')
new_df = pd.DataFrame(columns=columns)

length = len(df)
start = time.time()
display = None


if not os.path.exists('exports/train_metadata.csv'):
    for index, row in df.iterrows():
        display = print_progress_bar(index, length, calculate_time_remaining(start, index, length), display)
        filename = row['filename']
        meta_data = get_audio_metadata(filename)
        new_row = {
            'filename': filename,
            'primary_label': row['primary_label'],
            'secondary_labels': row['secondary_labels'],
            'type': row['type'],
            'collection': row['collection'],
            'rating': row['rating'],
            'url': row['url'],
            'latitude': row['latitude'],
            'longitude': row['longitude'],
            'scientific_name': row['scientific_name'],
            'common_name': row['common_name'],
            'author': row['author'],
            'license': row['license'],
            'sample_rate': meta_data['sample_rate'],
            'bit_rate': meta_data['bit_rate'],
            'sample_fmt': meta_data['sample_fmt'],
            'duration': meta_data['duration'],
            'num_channels': meta_data['num_channels'],
            'file_codec': meta_data['file_codec'],
            'file_codec_long_name': meta_data['file_codec_long_name'],
            'array_size': meta_data['array_size']
        }
        new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
    new_df.to_csv('exports/train_metadata.csv', index=False)
else:
    new_df = pd.read_csv('exports/train_metadata.csv')

In [26]:
new_df

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,sample_rate,bit_rate,sample_fmt,duration,num_channels,file_codec,file_codec_long_name,array_size
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62385,fltp,98853,1,vorbis,Vorbis,3163308
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62574,fltp,96538,1,vorbis,Vorbis,3089207
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,61940,fltp,116600,1,vorbis,Vorbis,3731194
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62444,fltp,105446,1,vorbis,Vorbis,3374282
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62875,fltp,103631,1,vorbis,Vorbis,3316207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28559,ywcpar,[''],[''],ywcpar/iNat77392.ogg,iNat,0.0,https://static.inaturalist.org/sounds/77392.m4a,7.6921,-80.3379,Amazona ochrocephala,Yellow-crowned Parrot,Yennifer Alfaro,cc-by-nc 4.0,32000,69806,fltp,3392,1,vorbis,Vorbis,108544
28560,ywcpar,[''],[''],ywcpar/iNat78624.ogg,iNat,0.0,https://static.inaturalist.org/sounds/78624.wav,8.9918,-79.4877,Amazona ochrocephala,Yellow-crowned Parrot,Evan Centanni,cc-by-nc-sa 4.0,32000,64741,fltp,7660,1,vorbis,Vorbis,245110
28561,ywcpar,[''],[''],ywcpar/iNat789234.ogg,iNat,0.0,https://static.inaturalist.org/sounds/789234.wav,9.2316,-70.2041,Amazona ochrocephala,Yellow-crowned Parrot,Henrry,cc-by 4.0,32000,65910,fltp,7667,1,vorbis,Vorbis,245333
28562,ywcpar,[''],[''],ywcpar/iNat819873.ogg,iNat,0.0,https://static.inaturalist.org/sounds/819873.mp3,10.5838,-66.8545,Amazona ochrocephala,Yellow-crowned Parrot,Alejandro Luy,cc-by-nc 4.0,32000,58547,fltp,27037,1,vorbis,Vorbis,865176


In [27]:
new_df['sample_rate'].value_counts()

sample_rate
32000    28564
Name: count, dtype: int64

In [28]:
new_df['bit_rate'].value_counts()

bit_rate
62880    12
63689    12
63738    11
61896    11
62588    11
         ..
56662     1
58749     1
55016     1
69805     1
59390     1
Name: count, Length: 12464, dtype: int64

In [29]:
new_df['sample_fmt'].value_counts()

sample_fmt
fltp    28564
Name: count, dtype: int64

In [30]:
new_df['file_codec'].value_counts()

file_codec
vorbis    28564
Name: count, dtype: int64

In [31]:
new_df['file_codec_long_name'].value_counts()

file_codec_long_name
Vorbis    28564
Name: count, dtype: int64

In [32]:
freq_bins = get_bark_spaced_frequencies(100, np.__name__) # Get the Bark spaced frequencies
new_columns = {f'{int(freq)}Hz': 0 for freq in freq_bins} # Create a dictionary with the new column names
new_columns_df = pd.DataFrame(new_columns, index=new_df.index) # Create a dataframe with the new columns
linear_df = pd.concat([new_df, new_columns_df], axis=1) # Concatenate the new columns to the dataframe
linear_df.head() # Display the first few rows of the dataframe

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,sample_rate,bit_rate,sample_fmt,duration,num_channels,file_codec,file_codec_long_name,array_size,20Hz,44Hz,69Hz,94Hz,119Hz,145Hz,170Hz,195Hz,221Hz,246Hz,272Hz,298Hz,324Hz,351Hz,377Hz,404Hz,432Hz,459Hz,487Hz,516Hz,544Hz,574Hz,603Hz,633Hz,664Hz,695Hz,727Hz,760Hz,793Hz,827Hz,861Hz,897Hz,933Hz,971Hz,1009Hz,1048Hz,1089Hz,1130Hz,1173Hz,1217Hz,1263Hz,1310Hz,1359Hz,1410Hz,1463Hz,1517Hz,1574Hz,1633Hz,1694Hz,1758Hz,1825Hz,1895Hz,1968Hz,2045Hz,2125Hz,2209Hz,2297Hz,2390Hz,2487Hz,2590Hz,2697Hz,2811Hz,2930Hz,3055Hz,3187Hz,3326Hz,3471Hz,3623Hz,3783Hz,3950Hz,4124Hz,4306Hz,4496Hz,4693Hz,4897Hz,5110Hz,5330Hz,5559Hz,5797Hz,6044Hz,6302Hz,6571Hz,6852Hz,7148Hz,7460Hz,7791Hz,8144Hz,8522Hz,8930Hz,9374Hz,9862Hz,10403Hz,11010Hz,11700Hz,12495Hz,13428Hz,14548Hz,15925Hz,17676Hz,19999Hz
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62385,fltp,98853,1,vorbis,Vorbis,3163308,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62574,fltp,96538,1,vorbis,Vorbis,3089207,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,61940,fltp,116600,1,vorbis,Vorbis,3731194,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62444,fltp,105446,1,vorbis,Vorbis,3374282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0,32000,62875,fltp,103631,1,vorbis,Vorbis,3316207,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We will eventually be splitting each file into a series of smaller files. In order to ensure we are not overfitting to the noise floor of each file, we will be splitting the dataset into a training and testing set before splitting each file into multiples so that the model has never seen the unique noise floors of the files.

In [33]:
train_linear_df, test_linear_df = train_test_split(linear_df, test_size=0.2) # Split the dataframe into training and testing sets
# reindex the dataframes
train_linear_df = train_linear_df.reset_index(drop=True) # Reset the index of the training dataframe
test_linear_df = test_linear_df.reset_index(drop=True) # Reset the index of the testing dataframe
pd.set_option('display.max_columns', None)
train_linear_df.head() # Display the first few rows of the training dataframe


,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,sample_rate,bit_rate,sample_fmt,duration,num_channels,file_codec,file_codec_long_name,array_size,20Hz,44Hz,69Hz,94Hz,119Hz,145Hz,170Hz,195Hz,221Hz,246Hz,272Hz,298Hz,324Hz,351Hz,377Hz,404Hz,432Hz,459Hz,487Hz,516Hz,544Hz,574Hz,603Hz,633Hz,664Hz,695Hz,727Hz,760Hz,793Hz,827Hz,861Hz,897Hz,933Hz,971Hz,1009Hz,1048Hz,1089Hz,1130Hz,1173Hz,1217Hz,1263Hz,1310Hz,1359Hz,1410Hz,1463Hz,1517Hz,1574Hz,1633Hz,1694Hz,1758Hz,1825Hz,1895Hz,1968Hz,2045Hz,2125Hz,2209Hz,2297Hz,2390Hz,2487Hz,2590Hz,2697Hz,2811Hz,2930Hz,3055Hz,3187Hz,3326Hz,3471Hz,3623Hz,3783Hz,3950Hz,4124Hz,4306Hz,4496Hz,4693Hz,4897Hz,5110Hz,5330Hz,5559Hz,5797Hz,6044Hz,6302Hz,6571Hz,6852Hz,7148Hz,7460Hz,7791Hz,8144Hz,8522Hz,8930Hz,9374Hz,9862Hz,10403Hz,11010Hz,11700Hz,12495Hz,13428Hz,14548Hz,15925Hz,17676Hz,19999Hz
0,butsal1,[''],['call'],butsal1/XC493406.ogg,XC,4.0,https://xeno-canto.org/493406,-6.3761,-47.1085,Saltator maximus,Buff-throated Saltator,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0,32000,59887,fltp,20950,1,vorbis,Vorbis,670407,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,banana,[''],['call'],banana/XC690554.ogg,XC,4.0,https://xeno-canto.org/690554,10.4370,-84.7093,Coereba flaveola,Bananaquit,Sue Riffe,cc-by-nc-sa 4.0,32000,65456,fltp,12800,1,vorbis,Vorbis,409600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,anhing,[''],['flight call'],anhing/XC482433.ogg,XC,1.0,https://xeno-canto.org/482433,-14.6250,-49.0051,Anhinga anhinga,Anhinga,JAYRSON ARAUJO DE OLIVEIRA,cc-by-nc-sa 4.0,32000,71509,fltp,4127,1,vorbis,Vorbis,132075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,orcpar,[''],['call'],orcpar/XC244355.ogg,XC,5.0,https://xeno-canto.org/244355,13.7333,-89.2167,Brotogeris jugularis,Orange-chinned Parakeet,Guillermo Funes,cc-by-nc-sa 4.0,32000,62693,fltp,51240,1,vorbis,Vorbis,1639680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,speowl1,[''],"['call', 'calls', 'immature']",speowl1/XC261645.ogg,XC,5.0,https://xeno-canto.org/261645,0.8830,-78.8000,Pulsatrix perspicillata,Spectacled Owl,Olaf Jahn,cc-by-nc-sa 4.0,32000,65141,fltp,33515,1,vorbis,Vorbis,1072483,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
start = time.time()
total_duration = train_linear_df['duration'].sum() # Calculate the total duration of the training dataframe
processed_duration = 0 # Initialize the processed duration
display = None # Initialize the display variable
new_df = pd.DataFrame(columns=train_linear_df.columns) # Create a new dataframe with the same columns as the training dataframe
if not os.path.exists('exports/train_linear_df.csv'):
    for index, row in train_linear_df.iterrows():
        display = print_progress_bar(processed_duration, total_duration, calculate_time_remaining(start, processed_duration, total_duration), display)
        filename = f'data/train_audio/{row['filename']}'
        sound = load_sound_file(filename)
        sounds = split_around_silence(sound)
        short_sounds = []
        for sound in sounds:
            sounds = window_sound(sound, length=5, overlap=2.5)
            for sound in sounds:
                short_sounds.append(sound)
        for sound in short_sounds:
            signal = sound.get_array_of_samples()
            signal = np.array(signal)
            signal = normalize_signal(signal)
            frequencies, freq_signal = fourier_transform(signal, sound.frame_rate)
            frequencies, freq_signal = group_frequencies(frequencies, freq_signal)
            freq_signal = magnitude_to_db(freq_signal)
            freq_signal = normalize_decibels(freq_signal)
            new_row = {
                'filename': row['filename'],
                'primary_label': row['primary_label'],
                'secondary_labels': row['secondary_labels'],
                'type': row['type'],
                'collection': row['collection'],    
                'rating': row['rating'],
                'url': row['url'],
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'scientific_name': row['scientific_name'],
                'common_name': row['common_name'],
                'author': row['author'],
                'license': row['license']
            }
            for i, freq in enumerate(freq_bins):
                new_row[f'{int(freq)}Hz'] = freq_signal[i]
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
        processed_duration += row['duration']
    new_df.to_csv('exports/train_linear_df.csv', index=False)

/tmp/ipykernel_153662/3589547729.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)


KeyboardInterrupt: 

In [ ]:
train_linear_df = pd.read_csv('exports/train_linear_df.csv') # Read the training dataframe from the CSV file
train_linear_df

In [ ]:
start = time.time()
total_duration = test_linear_df['duration'].sum() # Calculate the total duration of the training dataframe
processed_duration = 0 # Initialize the processed duration
display = None # Initialize the display variable
new_df = pd.DataFrame(columns=test_linear_df.columns) # Create a new dataframe with the same columns as the training dataframe
if not os.path.exists('exports/test_linear_df.csv'):
    for index, row in test_linear_df.iterrows():
        display = print_progress_bar(processed_duration, total_duration, calculate_time_remaining(start, processed_duration, total_duration), display)
        filename = f'data/train_audio/{row['filename']}'
        sound = load_sound_file(filename)
        sounds = split_around_silence(sound)
        short_sounds = []
        for sound in sounds:
            sounds = window_sound(sound, length=5, overlap=2.5)
            for sound in sounds:
                short_sounds.append(sound)
        print(f'Processing {len(short_sounds)} sounds')
        for sound in short_sounds:
            signal = sound.get_array_of_samples()
            signal = np.array(signal)
            signal = normalize_signal(signal)
            frequencies, freq_signal = fourier_transform(signal, sound.frame_rate)
            frequencies, freq_signal = group_frequencies(frequencies, freq_signal)
            freq_signal = magnitude_to_db(freq_signal)
            freq_signal = normalize_decibels(freq_signal)
            new_row = {
                'filename': row['filename'],
                'primary_label': row['primary_label'],
                'secondary_labels': row['secondary_labels'],
                'type': row['type'],
                'collection': row['collection'],    
                'rating': row['rating'],
                'url': row['url'],
                'latitude': row['latitude'],
                'longitude': row['longitude'],
                'scientific_name': row['scientific_name'],
                'common_name': row['common_name'],
                'author': row['author'],
                'license': row['license']
            }
            for i, freq in enumerate(freq_bins):
                new_row[f'{int(freq)}Hz'] = freq_signal[i]
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
        processed_duration += row['duration']
    new_df.to_csv('exports/test_linear_df.csv', index=False)

In [ ]:
test_linear_df = pd.read_csv('exports/train_linear_df.csv') # Read the training dataframe from the CSV file
test_linear_df